# Análisis Exploratorio de Datos (EDA) Clínico - NHANES

Este notebook realiza un análisis exploratorio detallado sobre el nuevo dataset clínico **NHANES**. 
El objetivo es validar la calidad de los biomarcadores, analizar sus distribuciones y entender su relación con la variable objetivo `HeartDisease`.

## Variables Clave
1. **Biomarcadores Numéricos:** SystolicBP, TotalCholesterol, LDL, Triglycerides, HbA1c, Glucose, UricAcid, Creatinine, WaistCircumference, BMI.
2. **Variables Categóricas:** Sex, Smoking, PhysicalActivity, HeartDisease (Target).

---

In [ ]:
# 1. Importación de Librerías
import pandas as pd
import sweetviz as sv
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración visual
sns.set_style("whitegrid")
pd.set_option('display.max_columns', None)

In [ ]:
# 2. Carga y Preparación de Datos
# Ruta al dataset procesado (ajustar según entorno)
DATA_PATH = "../data/02_intermediate/process_data.parquet"

try:
    df = pd.read_parquet(DATA_PATH)
    print(f"Dataset cargado. Dimensiones: {df.shape}")
except FileNotFoundError:
    print(f"ERROR: No se encontró el archivo en {DATA_PATH}. Verifica que el pipeline de datos se haya ejecutado.")
    # En caso de emergencia, intentar ruta absoluta o alternativa si existe
    # df = pd.read_parquet("data/02_intermediate/process_data.parquet")

# Mapeo de columnas (Español -> Inglés) para estandarizar reporte
column_mapping = {
    'Presion_Sistolica': 'SystolicBP',
    'Colesterol_Total': 'TotalCholesterol',
    'Trigliceridos': 'Triglycerides',
    'Glucosa': 'Glucose',
    'Acido_Urico': 'UricAcid',
    'Creatinina': 'Creatinine',
    'Cintura': 'WaistCircumference',
    'Sexo': 'Sex',
    'Fumador': 'Smoking',
    'Actividad_Fisica': 'PhysicalActivity',
    'TARGET': 'HeartDisease'
}

df_renamed = df.rename(columns=column_mapping)

# Definición de variables de interés
biomarkers = [
    'SystolicBP', 'TotalCholesterol', 'LDL', 'Triglicerides', 
    'HbA1c', 'Glucose', 'UricAcid', 'Creatinine', 
    'WaistCircumference', 'BMI'
]

categorical_vars = ['Sex', 'Smoking', 'PhysicalActivity', 'HeartDisease']

# Verificación de existencia de columnas
available_cols = df_renamed.columns.tolist()
missing = [col for col in biomarkers + categorical_vars if col not in available_cols]

if missing:
    print(f"\nADVERTENCIA: Las siguientes variables esperadas NO están en el dataset: {missing}")
    # Actualizamos la lista de biomarcadores a lo que sí existe para evitar errores
    biomarkers = [b for b in biomarkers if b in available_cols]
    categorical_vars = [c for c in categorical_vars if c in available_cols]

print("\nColumnas listas para análisis:")
print(df_renamed.columns.tolist())

### 3. Validación de Calidad de Datos (Biomarcadores)
Revisamos estadísticas descriptivas para detectar valores atípicos imposibles (ej. Glucosa negativa o Presión 0).

In [ ]:
print("--- Estadísticas Descriptivas (Biomarcadores) ---")
desc_stats = df_renamed[biomarkers].describe().T
display(desc_stats)

# Validación rápida de rangos fisiológicos mínimos
print("\n--- Chequeo de Valores <= 0 (Posibles Errores) ---")
potential_errors = {}
for b in biomarkers:
    # Asumimos que la mayoría de estos biomarcadores deben ser > 0
    count_zeros = df_renamed[df_renamed[b] <= 0].shape[0]
    if count_zeros > 0:
        potential_errors[b] = count_zeros

if potential_errors:
    print(f"¡ALERTA! Se detectaron valores <= 0 en: {potential_errors}")
else:
    print("Validación exitosa: Todos los biomarcadores tienen valores positivos.")

### 4. Análisis de Correlación
Calculamos la correlación de Pearson para detectar colinealidad entre biomarcadores.

In [ ]:
if len(biomarkers) > 1:
    plt.figure(figsize=(12, 10))
    corr_matrix = df_renamed[biomarkers].corr()
    
    sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='RdBu_r', vmin=-1, vmax=1)
    plt.title("Matriz de Correlación de Biomarcadores Clínicos")
    plt.show()
    
    # Identificar correlaciones altas
    print("Parejas con correlación absoluta > 0.7:")
    corr_pairs = corr_matrix.unstack().sort_values(key=abs, ascending=False)
    # Filtramos la diagonal (correlación 1.0) y duplicados
    high_corr = corr_pairs[(abs(corr_pairs) > 0.7) & (abs(corr_pairs) < 1.0)]
    if not high_corr.empty:
        print(high_corr[::2]) # Imprimimos cada 2 para evitar (A,B) y (B,A)
    else:
        print("No se detectaron correlaciones lineales fuertes (>0.7).")
else:
    print("Insuficientes biomarcadores para análisis de correlación.")

### 5. Generación de Reporte Visual (Sweetviz)
Generamos un reporte HTML comparativo separando la población por `HeartDisease` (Target).

In [ ]:
REPORT_FILENAME = "NHANES_Clinical_Analysis.html"

# Configuración de features para asegurar correcta interpretación
# Forzamos categóricas si tienen pocos valores únicos numéricos, aunque Sweetviz es inteligente
# Aseguramos que el Target sea tratado adecuadamente
feature_config = sv.FeatureConfig(force_cat=categorical_vars)

print(f"Iniciando generación de reporte Sweetviz... (Target: HeartDisease)")

if 'HeartDisease' in df_renamed.columns:
    analysis = sv.analyze(
        [df_renamed, "NHANES Data"],
        target_feat='HeartDisease',
        feat_cfg=feature_config,
        pairwise_analysis='off' # Apagado para eficiencia, ya hicimos correlación arriba
    )
    
    analysis.show_html(filepath=REPORT_FILENAME)
    print(f"\n¡Reporte generado exitosamente! Abrir archivo: {REPORT_FILENAME}")
else:
    print("ERROR CRÍTICO: No se encuentra la columna 'HeartDisease' para el análisis comparativo.")
    # Fallback sin target
    analysis = sv.analyze([df_renamed, "NHANES Data"], pairwise_analysis='off')
    analysis.show_html(filepath=REPORT_FILENAME)